In [1]:
! pip install -q ragstack-ai datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.4/79.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 54.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.0/990.0 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 84.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import cassio

In [8]:
import os
from getpass import getpass

# Enter your settings for Astra DB and OpenAI:
os.environ["ASTRA_DB_API_ENDPOINT"] ="https://76e048f3-cea2-40c6-b2d2-61be6361528b-us-east-2.apps.astra.datastax.com"
os.environ["ASTRA_DB_APPLICATION_TOKEN"] ="AstraCS:eBciXcyXCnbdoovZlCMuehSP:aa770408bdae9e8bcfb16865d3615aa754e13faaf6a5254653aede94978b6584"
os.environ["OPENAI_API_KEY"] = getpass("Open api key")

Open api key··········


In [9]:
from langchain_astradb import AstraDBVectorStore
from langchain.embeddings import OpenAIEmbeddings
import os

# Configure your embedding model and vector store
embedding = OpenAIEmbeddings()
vstore = AstraDBVectorStore(
    collection_name="QACB",
    embedding=embedding,
    token=os.getenv("ASTRA_DB_APPLICATION_TOKEN"),
    api_endpoint=os.getenv("ASTRA_DB_API_ENDPOINT"),
)
print("Astra vector store configured")

Astra vector store configured


In [15]:
from datasets import load_dataset

# Load the dataset from the Hugging Face Hub
movie_data = load_dataset("scoup123/tr_movie_reviews_training", split="train")

print("An example entry:")
print(movie_data[16])


Generating train split:   0%|          | 0/66582 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/16645 [00:00<?, ? examples/s]

An example entry:
{'Unnamed: 0': 16, 'Sentence': ' nerdeyim yahu.. uzun zamandır yokum ortalıklarda ', 'Sentiment': 'Positive'}


In [19]:
from langchain.schema import Document

# Create a list to hold the constructed Document objects
docs = []

# Iterate over each entry in the dataset
for entry in movie_data:
    # Initialize the metadata dictionary with the Sentiment information
    metadata = {"Sentiment": entry["Sentiment"]}

    # Check if the 'Unnamed: 0' column has any value
    if entry["Unnamed: 0"] is not None:
        # Split the 'Unnamed: 0' field by ";" and add each as a metadata tag
        for tag in str(entry["Unnamed: 0"]).split(";"):
            metadata[tag.strip()] = "y"

    # Create a Document object with the sentence and its metadata
    doc = Document(page_content=entry["Sentence"], metadata=metadata)

    # Add the Document object to the list
    docs.append(doc)


In [ ]:
# Create embeddings by inserting your documents into the vector store.
inserted_ids = vstore.add_documents(docs)
print(f"\nInserted {len(inserted_ids)} documents.")

In [ ]:
# Checks your collection to verify the documents are embedded.
print(vstore.astra_db.collection("test").find())

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

retriever = vstore.as_retriever(search_kwargs={"k": 3})

prompt_template = """
Answer the question based only on the supplied context. If you don't know the answer, say you don't know the answer.
Context: {context}
Question: {question}
Your answer:
"""
prompt = ChatPromptTemplate.from_template(prompt_template)
model = ChatOpenAI()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke("In the given context, what is the review which has the word nolcağını?")